# Comparing different algorithms

In [ ]:
#SLPA
from cdlib import algorithms,evaluation as ev,readwrite
import networkx as nx
import pandas as pd
import numpy as np
import time
import os

def compare_slpa(g,f):
    e=Emodularity()
    r_l=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    m_score=[]
    for i in r_l:
        i_j={}
        for j in range(3):
            c={}
            st=time.time()
            c_cdlib=algorithms.slpa(g,t=30,r=i)
            sp=time.time()
            readwrite.write_community_csv(c_cdlib,f+str(i)+'_'+str(j)+'.csv')
            m_cdlib=ev.link_modularity(g,c_cdlib).score
            p=ev.conductance(g,c_cdlib).score
            o=e.emodularity(g,c_cdlib)
            print(p)
            print(m_cdlib)
            c['link_modularity']=m_cdlib
            c['conduance']=p
            c['over_modularity']=o
            c['time']=sp-st
            c['com_count']=len(c_cdlib.communities)
            c['iter']=j
            i_j[j]=c
        dt=pd.DataFrame(i_j)
        dt1 = dt.stack()
        dt2 = dt1.unstack(0)
        dt2['r']=[i]*len(dt2.index)
        dt2.to_excel(f + 'calculate_output_{}.xlsx'.format(i), index=True, header=True)
        m_score.append(dt2)
    m_score=pd.concat(m_score)
    m_score.to_excel(f+'all_cal.xlsx')
def make_filedir(filepath):
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    return filepath
fo='F:/aps_analysis/aps_compare_slpa/'
fi='F:/aps_out/aps/APS_network/aps_article_network/article_journal_network/'
# g_list=['PRSTPER','PRI','PRMATERIALS','RMP','PRAPPLIED','PRX','PRSTAB','PRPER','PRFLUIDS','PRAB','PRE','PR',]
# ,'PRC','PRA','PRL','PRD'
g_list = ['PRB']
for g in g_list:
    f=make_filedir(fo+g+'/')
    G_file=fi+g+'.gexf'
    G = nx.read_gexf(G_file, node_type=str)
    # 判断图的类型
    if nx.is_directed(G):
        G = G.to_undirected()
    s=list(nx.isolates(G))
    G.remove_nodes_from(s)
    compare_slpa(G,f)



In [ ]:
#DEMO
#比较demo
from cdlib import algorithms,evaluation as ev,readwrite
import networkx as nx
import pandas as pd
import numpy as np
import time
import os
class Emodularity():
    def emodularity(self, G, partition):
        #com_node: {com:[node_id]}, neigh:  {node_id:[node_id,node_id]}, node_com  {node_id:[com_id]}
        comm=partition.communities
        com_node=self.get_com_node(comm)
        node_com=self.com_node_reversal(com_node)
        neigh=self.Nodeneigh(G)
        com = com_node.keys()
        m = float(G.number_of_edges())
        score = 0
        error_node = 0

        for c in com:
            for v in com_node[c]:
                  for u in com_node[c]:
                    if v == u:
                        pass
                    else:
                        try:
                            if u in neigh[v]:
                                a = 1
                            else:
                                a = 0
                            ov = len(node_com[v])
                            ou = len(node_com[u])
                            kv = len(neigh[v])
                            ku = len(neigh[u])
                            score = score + (1 / (ov * ou) * (a - (kv * ku) / (2 * m)))
                        except Exception as e:
                            print("节点{}错误".format(v))
                            error_node = error_node + 1
                            continue
                        else:
                            pass
        print("错误节点数{}".format(error_node))
        score = float(score / (2 * m))
        return score
    def get_com_node(self,partiton):
        com_node={}
        for i ,j in enumerate(partiton):
            com_node[i]=j
        return com_node
    def com_node_reversal(self,com_node):
        node_com = {}
        print("转化为node_community形式")
        for k ,v in com_node.items():
            for node in v:
                s=[]
                if node in node_com.keys():
                    if k in node_com[node]:
                        pass
                    else:
                        s = node_com[node]
                        node_com[node] = s + [k]


                else:
                    s=[k]
                    node_com[node]=s
        return node_com
    def Nodeneigh(self,G):
        node_attr={}
        for n in list(G.nodes()):
            nei=list(G.neighbors(n))
            # nei=list(G.successors(n))+list(G.predecessors(n))
            # print(len(nei))
            # n=int(n)
            node_attr[n]=nei
        return node_attr

def compare_demo(G,f):
    e=Emodularity()
    r_l=[0.25,0.5]
    m_score=[]
    for i in r_l:
        i_j={}
        for j in range(1):
            g=G.copy()
            c={}
            st=time.time()
            c_cdlib=algorithms.demon(g,epsilon=i)
            sp=time.time()
            readwrite.write_community_csv(c_cdlib,f+str(i)+'_'+str(j)+'.csv')
            m_cdlib=ev.newman_girvan_modularity(g,c_cdlib).score
            # p=ev.conductance(g,c_cdlib).score
            o=e.emodularity(g,c_cdlib)
            # print(p)
            # print(m_cdlib)
            c['modularity']=m_cdlib
            # c['conduance']=p
            c['over_modularity']=o
            c['time']=sp-st
            c['com_count']=len(c_cdlib.communities)
            c['iter']=j
            print(c)
            i_j[j]=c
        dt=pd.DataFrame(i_j)
        dt1 = dt.stack()
        dt2 = dt1.unstack(0)
        dt2['r']=[i]*len(dt2.index)
        dt2.to_excel(f + g.name+'_calculate_output_{}.xlsx'.format(i), index=True, header=True)
        m_score.append(dt2)
    m_score=pd.concat(m_score)
    m_score.to_excel(f+'all_cal.xlsx')
def make_filedir(filepath):
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    return filepath

fo='work/lfm/'
fi='work/aps/aps/APS_network/aps_article_network/article_journal_network/'
g_list=['PRSTPER','PRI','PRMATERIALS','RMP','PRAPPLIED','PRX','PRSTAB','PRPER','PRFLUIDS','PRAB','PRE','PR','PRB','PRC','PRA','PRL','PRD']
#
# g_list = []
for g in g_list:
    f=make_filedir(fo+g+'/')
    G_file=fi+g+'.gexf'
    G = nx.read_gexf(G_file, node_type=str)
    # 判断图的类型
    if nx.is_directed(G):
        G = G.to_undirected()
    s=list(nx.isolates(G))
    G.remove_nodes_from(s)
    compare_demo(G,f)


In [ ]:
#prepare for SCI, the original algorithm written by should refer to 
#Wang, X., Jin, D., Cao, X., Yang, L., & Zhang, W. (2016). Semantic community identification in large attribute networks. In Proceedings of the Thirtieth AAAI Conference on Artificial Intelligence (AAAI’16). AAAI Press, 265–271.
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import scipy.io as scio
import numpy as np
import pandas as pd
import networkx as nx
import json
import pandas as pd
from cdlib import *
from cdlib import evaluation,readwrite
def read_edge(fe):
    edgelist=[]
    with open(fe,'w',encoding='utf-8')as f:
        for e in f.readlines():
            e=e.strip('\n').split(' ')
            edgelist.append(e)
    edgelist=np.asarray(edgelist)
    return edgelist
def read_sentence(fv):
    sen={}
    with open(fv,'r',encoding='utf-8')as f:
        for w in f.readlines():
            w=w.strip('\n').split(':')
            sen[w[0]]=w[1]
    return sen
def word_vector(texts):
    cv = CountVectorizer(binary=True)
    document_vec = cv.fit_transform(texts)
    v=document_vec.toarray()
    return v
def main():
    f_g=''
    f_v=''
    f_s=''
    f_gs=''
    # 'PRMATERIALS',
    g_list = ['PRSTPER', 'PRI', 'RMP', 'PRAPPLIED', 'PRX', 'PRSTAB',
              'PRPER', 'PRFLUIDS', 'PRAB', 'PRE', 'PR', 'PRB', 'PRC', 'PRA', 'PRL', 'PRD']

    for g in g_list:
        data={}
        fg=f_g+g+'.gexf'
        fv=f_v+g+'.txt'
        data_save=f_s+g+'.mat'
        fgs=f_gs+g+'.gexf'
        G=nx.read_gexf(fg)
        G1=nx.convert_node_labels_to_integers(G,first_label=1,label_attribute=True)
        edge_list=[]
        l=nx.selfloop_edges(G1)
        G1.remove_edges_from(l)
        nx.write_gexf(G1,fgs)
        for line in nx.generate_edgelist(G1, data=False):
            edge_list.append(list(line))
        edge_list=np.asarray(edge_list)
        data['adj']=edge_list
        sen=read_sentence(fv)
        sen_list=[]
        for n in list(G1.nodes()):
            n_label=G1.nodes[n]['label_attribute']
            sen_list.append(sen[n_label])
        content_vector=word_vector(sen_list)
        data['content']=content_vector
        scio.savemat(data_save,data)
def cal_sci_Q():
    g_list = ['PRSTPER', 'PRI', 'RMP', 'PRAPPLIED', 'PRX', 'PRSTAB',
              'PRPER', 'PRFLUIDS', 'PRAB', 'PRE', 'PR', 'PRB', 'PRC', 'PRA', 'PRL', 'PRD']
    f=''
    f_g=''
    f_o=''
    calcul={}
    for g in g_list:
        cal={}
        fn=f+g+'_output.mat'
        fg=f_g+g+'.gexf'
        G=nx.read_gexf(fg)
        data=scio.loadmat(fn)
        # Ucontent, res, U1index
        com=data['res']
        comms={}
        nodes={}
        for n,c in enumerate(com):
            nodes[n+1]=c
        for k ,v in nodes.items():
            if v in comms.keys():
                comms[v].append(k)
            else:
                comms[v]=[k]
        com_size={}
        for i,j in comms.items():
            com_size[i]=len(j)
        with open(f_o+g+'_comsize.json','w',encoding='utf-8')as f1:
            json.dump(com_size,f1)
        community=list(comms.values())
        partition= NodeClustering(community, G, "SCI", method_parameters={"T": 100, "r": 0.25}, overlap=False)
        readwrite.write_community_csv(partition,f_o+g+'_com.csv')
        cal['modularity']=evaluation.newman_girvan_modularity(G,partition).score
        cal['com']=len(comms)
        calcul[g]=cal
    calcul=pd.DataFrame(calcul)
    calcul.to_excel(f_o+'calculate_sci.xlsx')
    print(calcul)


In [ ]:
#Infomap refer to mapequation.org/infomap/#Install
#OSLOM refer to oslom.org

In [ ]:
#LPA
from cdlib import algorithms,evaluation as ev,readwrite
import networkx as nx
import pandas as pd
import numpy as np
import time
import os
def compare_lpa(G,f):
    i_j=[]
    for j in range(10):
        g=G.copy()
        c={}
        st=time.time()
        c_cdlib=algorithms.label_propagation(g)
        sp=time.time()
        readwrite.write_community_csv(c_cdlib,f+'_'+str(j)+'.csv')
        m_cdlib=ev.newman_girvan_modularity(g,c_cdlib).score
        p=ev.conductance(g,c_cdlib).score
        print(p)
        print(m_cdlib)
        c['modularity']=m_cdlib
        c['conduance']=p
        c['time']=sp-st
        c['com_count']=len(c_cdlib.communities)
        c['iter']=j
        i_j.append(c)
    dt=pd.DataFrame(i_j)
    dt.to_excel(f + '{}.xlsx'.format(g.name), index=True, header=True)
def make_filedir(filepath):
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    return filepath
fo='F:/aps_analysis/lpa/'
fi= 'F:/aps_out/aps/APS_network/aps_article_network/article_journal_network/'
g_list=['PRSTPER','PRI','PRMATERIALS','RMP','PRAPPLIED','PRX','PRSTAB','PRPER','PRFLUIDS','PRAB','PRE','PR','PRB','PRC','PRA','PRL','PRD']
# 'PRB',
# g_list = []
for g in g_list:
    f=make_filedir(fo+g+'/')
    G_file=fi+g+'.gexf'
    G = nx.read_gexf(G_file, node_type=str)
    # 判断图的类型
    if nx.is_directed(G):
        G = G.to_undirected()
    s=list(nx.isolates(G))
    G.remove_nodes_from(s)
    compare_lpa(G,f)


In [ ]:
class Emodularity():
    def emodularity(self, G, partition):
        #com_node: {com:[node_id]}, neigh:  {node_id:[node_id,node_id]}, node_com  {node_id:[com_id]}
        comm=partition.communities
        com_node=self.get_com_node(comm)
        node_com=self.com_node_reversal(com_node)
        neigh=self.Nodeneigh(G)
        com = com_node.keys()
        m = float(G.number_of_edges())
        score = 0
        error_node = 0

        for c in com:
            for v in com_node[c]:
                  for u in com_node[c]:
                    if v == u:
                        pass
                    else:
                        try:
                            if u in neigh[v]:
                                a = 1
                            else:
                                a = 0
                            ov = len(node_com[v])
                            ou = len(node_com[u])
                            kv = len(neigh[v])
                            ku = len(neigh[u])
                            score = score + (1 / (ov * ou) * (a - (kv * ku) / (2 * m)))
                        except Exception as e:
                            print("节点{}错误".format(v))
                            error_node = error_node + 1
                            continue
                        else:
                            pass
        print("错误节点数{}".format(error_node))
        score = float(score / (2 * m))
        return score
    def get_com_node(self,partiton):
        com_node={}
        for i ,j in enumerate(partiton):
            com_node[i]=j
        return com_node
    def com_node_reversal(self,com_node):
        node_com = {}
        print("转化为node_community形式")
        for k ,v in com_node.items():
            for node in v:
                s=[]
                if node in node_com.keys():
                    if k in node_com[node]:
                        pass
                    else:
                        s = node_com[node]
                        node_com[node] = s + [k]


                else:
                    s=[k]
                    node_com[node]=s
        return node_com
    def Nodeneigh(self,G):
        node_attr={}
        for n in list(G.nodes()):
            nei=list(G.neighbors(n))
            # nei=list(G.successors(n))+list(G.predecessors(n))
            # print(len(nei))
            # n=int(n)
            node_attr[n]=nei
        return node_attr

In [ ]:
#iLOUVAIN
#ilouvain
import random
from cdlib import algorithms,evaluation as ev,readwrite
import networkx as nx
import pandas as pd
import numpy as np
import time
import os
class Emodularity():
    def emodularity(self, G, partition):
        #com_node: {com:[node_id]}, neigh:  {node_id:[node_id,node_id]}, node_com  {node_id:[com_id]}
        comm=partition.communities
        com_node=self.get_com_node(comm)
        node_com=self.com_node_reversal(com_node)
        neigh=self.Nodeneigh(G)
        com = com_node.keys()
        m = float(G.number_of_edges())
        score = 0
        error_node = 0

        for c in com:
            for v in com_node[c]:
                  for u in com_node[c]:
                    if v == u:
                        pass
                    else:
                        try:
                            if u in neigh[v]:
                                a = 1
                            else:
                                a = 0
                            ov = len(node_com[v])
                            ou = len(node_com[u])
                            kv = len(neigh[v])
                            ku = len(neigh[u])
                            score = score + (1 / (ov * ou) * (a - (kv * ku) / (2 * m)))
                        except Exception as e:
                            print("节点{}错误".format(v))
                            error_node = error_node + 1
                            continue
                        else:
                            pass
        print("错误节点数{}".format(error_node))
        score = float(score / (2 * m))
        return score
    def get_com_node(self,partiton):
        com_node={}
        for i ,j in enumerate(partiton):
            com_node[i]=j
        return com_node
    def com_node_reversal(self,com_node):
        node_com = {}
        print("转化为node_community形式")
        for k ,v in com_node.items():
            for node in v:
                s=[]
                if node in node_com.keys():
                    if k in node_com[node]:
                        pass
                    else:
                        s = node_com[node]
                        node_com[node] = s + [k]


                else:
                    s=[k]
                    node_com[node]=s
        return node_com
    def Nodeneigh(self,G):
        node_attr={}
        for n in list(G.nodes()):
            nei=list(G.neighbors(n))
            # nei=list(G.successors(n))+list(G.predecessors(n))
            # print(len(nei))
            # n=int(n)
            node_attr[n]=nei
        return node_attr

def read_att(fi):
    data={}
    with open(fi,'r',encoding='utf-8')as f:
        for c in f.readlines():
            c=c.strip('\n').split(':')
            #由于原始数据存入时有bug,多写了一个join，因此造成每个数字间有三个空格。在此处需要注意
            l=c[1].split('   ')
#             print(l)
            l=list(map(float,l))
            dt={}
            for i,j in enumerate(l):
#                 print(type(j))
                dt[i]=j
            data[c[0]]=dt
    return data

def compare_ilouvain(G,label,f):
    labels={}
    node_id={}
    #防止某些节点不在label中
    for n in list(G.nodes()):
        labels[n]=label[n]
        node_id[n]=n
    
    
    e=Emodularity()
#     r_l=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    r_l=[1]
    m_score=[]
    for i in r_l:
        i_j={}
        for j in range(1):
            g=G.copy()
            c={}
            st=time.time()
            c_cdlib=algorithms.ilouvain(g, labels, node_id)
            sp=time.time()
            readwrite.write_community_csv(c_cdlib,f+str(g.name)+'_'+str(i)+'.csv')
            m_cdlib=ev.link_modularity(g,c_cdlib).score
            p=ev.newman_girvan_modularity(g,c_cdlib).score
            o=e.emodularity(g,c_cdlib)
            print(o)
#             print(m_cdlib)
            c['link_modularity']=m_cdlib
            c['modularity']=p
            c['over_modularity']=o
            c['time']=sp-st
            c['com_count']=len(c_cdlib.communities)
            c['iter']=j
            print(c)
            i_j[j]=c
        dt=pd.DataFrame(i_j)
        dt1 = dt.stack()
        dt2 = dt1.unstack(0)
        dt2['r']=[i]*len(dt2.index)
        dt2.to_excel(f + '{}_{}_cal.xlsx'.format(g.name,i), index=True, header=True)
        m_score.append(dt2)
    m_score=pd.concat(m_score)
    m_score.to_excel(f+'all_cal.xlsx')
def make_filedir(filepath):
    if not os.path.exists(filepath):
        os.mkdir(filepath)
    return filepath  
fgs='F:/aps_analysis/aps_compare_dcm/graph/'
fts='F:/aps_analysis/aps_compare_dcm/count_dict/'
fo='F:/aps_analysis/aps_compare_ilouvain/'
g_list=[
# 'PRPER','PRAB','PRFLUIDS','PRI','PRSTPER','PRAPPLIED','PRX',
        'PRSTAB'
#         ,'RMP','PRE','PR','PRC','PRA','PRD','PRL','PRB'
       ]
# g_list=['PRSTPER']
for g in g_list:
    print(g)
    ft=fts+g+'_count_dict.dsm'
    fg=fgs+g+'.gexf'
    node_label=read_att(ft)
    G = nx.read_gexf(fg)
    # 判断图的类型,G在上一步已经去掉self_loop,节点标签转换为1开始的数字，不清楚是否有向变无向
    #和去掉度为零的节点，因此在这里重新处理
    if nx.is_directed(G):
        G = G.to_undirected()
    s=list(nx.isolates(G))
    G.remove_nodes_from(s)
    compare_ilouvain(G,node_label,fo)

In [ ]:
import pandas as pd
import numpy as np
import os
from cdlib import evaluation, readwrite
import networkx as nx
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plta
from cdlib import  algorithms,evaluation
from cdlib import *
import json

"""需要获取节点的邻居节点，节点度，节点拥有的社区，节点的社区数"""
class EQ():
    def __init__(self,filepath):
        self.filepath=filepath
    def Nodeneigh(self,G):
        node_attr={}
        print("读取节点邻居节点")
        for n in list(G.nodes()):
            # print("节点{}的度为{}".format(n,G.degree(n)))
            # print(list(G.successors(n)))
            # print(list(G.neighbors(n)))
            # print(list(G.predecessors(n)))
            nei=list(G.neighbors(n))
            # nei=list(G.successors(n))+list(G.predecessors(n))
            # print(len(nei))
            # n=int(n)
            # node_attr[n]=list(map(int,nei))
            node_attr[n] =  nei
        # print(node_attr)
        return node_attr
    def read_comnode(self,filename):
        print("读取社区划分结果")
        com_node = {}
        node_list=[]
        i = 0
        with open(filename, 'r', encoding='utf8') as f:
            for r in f.readlines():
                r = r.strip().strip('\n').strip('\t')
                if r is not None:
                    n_list=list(map(int, r.split()))
                    com_node[i] = n_list
                    node_list=node_list+n_list
                    i = i + 1
                else:
                    continue
        print("社区数量{}".format(i))

        return com_node
    def com_node_reversal(self,com_node):
        node_com = {}
        print("转化为node_community形式")
        for k ,v in com_node.items():
            for node in v:
                s=[]
                if node in node_com.keys():
                    if k in node_com[node]:
                        pass
                    else:
                        s = node_com[node]
                        node_com[node] = s + [k]


                else:
                    s=[k]
                    node_com[node]=s
        return node_com
    def Emodularity(self,G,com_node,n_attr):
        print("计算EQ")
        print("图中节点数{}".format(len(n_attr)))
        # com_node=self.read_comnode(filename)

        node_com=self.com_node_reversal(com_node)
        print("社团划分包括节点数{}".format(len(node_com.keys())))
        com=com_node.keys()
        m=float(G.number_of_edges())
        nodes=list(node_com.keys())

        print("交集数量为")
        print(len(list(set(n_attr.keys()) & set(nodes))))
        node_attr = n_attr
        score=0
        error_node=0
        count=0
        for c in com:
            print("社区{}".format(c))
            for v in com_node[c]:
                count=count+1
                print("开始遍历第{}个节点,还有{}个节点".format(count,(len(nodes)-count)))
                for u in com_node[c]:
                    if v==u:
                        pass
                    else:
                        # print(node_attr[v])
                        try:
                            if u in node_attr[v]:
                                a = 1
                            else:
                                a = 0
                            ov = len(node_com[v])
                            ou = len(node_com[u])
                            kv = len(node_attr[v])
                            ku = len(node_attr[u])
                            score = score + (1 / (ov * ou) * (a - (kv * ku) / (2 * m)))
                        except Exception as e:
                                print("节点{}错误".format(v))
                                error_node=error_node+1
                                continue
                        else:
                            pass
        print("错误节点数{}".format(error_node))
        score=float(score/(2*m))
        return score
    def score_number(self,g_list):
        
        f_g='F:/aps_analysis/graph/edgelist_int/'
        f_c='F:/aps_analysis/aps_compare_oslom/COM/'
        f_o='F:/aps_analysis/aps_compare_oslom/output/'
        d={}
        for g in g_list:
            c={}
            fg=f_g+g+'.gexf'
            fc=f_c+g
            fo=f_o+g+'community.txt'
            G=nx.read_gexf(fg)
            node_attr=self.Nodeneigh(G)

            com,com_size=self.read_infomap_com(fc)
            self.write_dict_list(com,fo)
            c['modularity']=self.Emodularity(G,com,node_attr)
            comms=list(com.values())

            partition = NodeClustering(comms, G, "infomap", method_parameters={"T": 100, "r": 0.25}, overlap=True)

            c['conduance']=evaluation.conductance(G,partition).score
            c['newman_modularity']=evaluation.newman_girvan_modularity(G,partition).score
            c['com_count']=len(com.keys())
            d[g]=c
            readwrite.write_community_csv(partition,fo+'community.csv')
            with open(fo+'_com_node_count.json','w',encoding='utf-8')as f:
                json.dump(com_size,f)
        data=pd.DataFrame(d)
        data.to_excel(f_o+'infomap_cal.xlsx')
        return data
    def read_G(self,f,textname):
        G = nx.DiGraph()
        l=[]
        with open(f+textname,'r')as f:
           for s in f.readlines():
               s=s.strip('\n').strip('\t')
               n=list(s.split(' '))
               l.append(n)

        G.add_weighted_edges_from(l)
        print(G.number_of_nodes())
        print(G.number_of_edges())
        return G

    def read_infomap_com(self,f):
        f=f+'.dat.clu'
        comms = {}
        com_size = {}
        with open(f , 'r', encoding='utf-8')as f1:
            i = 0
            for c in f1.readlines():
                i = i + 1
                if i < 10:
                    pass
                else:
                    node = c.strip('\n').split(' ')[0]
                    com = c.strip('\n').split(' ')[1]
                    if com in comms.keys():
                        comms[com].append(node)
                    else:
                        comms[com] = [node]
            comms = dict(sorted(comms.items(), reverse=False, key=lambda k: k[1]))
            for k,v in comms.items():
                com_size[k]=len(v)
            # self.write_dict_list(com_size,f+s+'_comsize.txt')


        return comms,com_size

    def read_com(self,fn):
        com = {}
        com_size = {}
        i = 0
        with open(fn, 'r', encoding='utf-8')as f:
            for c in f.readlines():

                if '#' in c:
                    pass
                else:
                    n = c.lstrip(' ').rstrip(' ').strip('\n').split(' ')
                    n=[k for k in n if k !='']
                    print(n)
                    com[i] = n
                    com_size[i] = len(n)
                    i = i + 1
        com = dict(sorted(com.items(), reverse=False, key=lambda k: k[1]))
        return com, com_size
    def write_dict_list(self,d_list,filename):
        with open(filename,'w',encoding='utf-8')as f:
            for k ,v in d_list.items():
                f.write(str(k)+':'+' '.join(v))
                f.write('\n')
        return


if __name__ == '__main__':
    g_list = ['PRSTPER', 'PRI', 'PRMATERIALS', 'RMP', 'PRAPPLIED', 'PRX', 'PRSTAB',
              'PRPER', 'PRFLUIDS', 'PRAB', 'PRE', 'PR', 'PRB', 'PRC', 'PRA', 'PRL', 'PRD']
    f_o = 'F:/aps_analysis/aps_compare_oslom/output/'
    e=EQ(f_o)
    e.score_number(g_list)


